<a href="https://colab.research.google.com/github/psahai10/AIFellowship/blob/main/Copy_of_Copy_of_Load_Custom_C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Custom-C Dataset

In [ ]:
# Mount Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# make directory to hold images
!mkdir /content/images/
# unzip and extract
!unzip -uq '/content/gdrive/MyDrive/Custom-C.zip' -d "/content/images"

In [ ]:
# extract image_urls and label_names
import os
image_urls = []
labels_dist = []

path = '/content/images/Bruh'
for f in os.listdir(path):
  if(f!='.DS_Store'):
    image = os.path.join(path, f)
    image_urls.append(image)
    f = f.split('.')[-2]
    stemmed = ''.join([i for i in f if not i.isdigit()])
    stemmed = stemmed.replace('()','')
    stemmed = stemmed.strip()
    labels_dist.append(stemmed)

print(f'# of images: {len(image_urls)}')
print(f'# of labels: {len(labels_dist)}')

# of images: 31490
# of labels: 31490


# Import Resources for Labelling

In [ ]:
# copy files from drive to current session
!cp /content/gdrive/MyDrive/Resources/labels_dict.py /content/
!cp /content/gdrive/MyDrive/Resources/labels_categories0308.csv /content/

In [ ]:
import pandas as pd
from labels_dict import *

In [ ]:
# store urls and raw labels in dataframe
import pandas as pd
df = pd.DataFrame(list(zip(image_urls,labels_dist)), columns = ['image_urls', 'labels_dist'])

In [ ]:
# clean label names
label_names = []
for i in labels_dist:
  if i in labels_dict:
    label_names.append(labels_dict[i].lower()) # use the pre-defined dictionary
  else:
    label_names.append(i)

# Add cleaned labels to df
df['labels_clean']= label_names
df.labels_clean = df.labels_clean.str.replace('_',' ')
df.drop(columns='labels_dist', inplace=True)
df.columns = ['image_urls','label']

In [ ]:
# load current labels/categories csv
csv_path = 'labels_categories0308.csv'
all_labels_cats = pd.read_csv(csv_path, encoding='UTF-8')

# remove extra columns
all_labels_cats = all_labels_cats[['label','category','common_in_america']]

# Create `main_df` 
  - `image_url`, `label`, `category`, `common_in_america`

In [ ]:
main_df = df.merge(all_labels_cats, how='left')
main_df.drop(main_df[main_df.category.isna()].index, inplace=True)
main_df.columns = ['image_url', 'label', 'category','common_in_america']
main_df.describe()

,image_url,label,category,common_in_america
count,24357,24357,24357,24357
unique,24165,257,53,94
top,/content/images/Bruh/gratin81.jpg,gratin,vegetable dish,cake
freq,2,222,2062,1678


In [ ]:
main_df.image_url.value_counts()

/content/images/Bruh/gratin81.jpg            2
/content/images/Bruh/tater tots27.jpg        2
/content/images/Bruh/gratin53.jpg            2
/content/images/Bruh/gratin37.jpg            2
/content/images/Bruh/gratin32.jpg            2
                                            ..
/content/images/Bruh/ayam_goreng68.jpg       1
/content/images/Bruh/lasagna71.jpg           1
/content/images/Bruh/red_velvet_cake9.jpg    1
/content/images/Bruh/lava_cake50.jpg         1
/content/images/Bruh/tsoureki68.jpg          1
Name: image_url, Length: 24165, dtype: int64

In [ ]:
import subprocess
 
CUDA_version = [s for s in subprocess.check_output(["nvcc", "--version"]).decode("UTF-8").split(", ") if s.startswith("release")][0].split(" ")[-1]
print("CUDA version:", CUDA_version)
 
if CUDA_version == "10.0":
    torch_version_suffix = "+cu100"
elif CUDA_version == "10.1":
    torch_version_suffix = "+cu101"
elif CUDA_version == "10.2":
    torch_version_suffix = ""
else:
    torch_version_suffix = "+cu110"

CUDA version: 11.0


In [ ]:
!pip install torch==1.8.0{torch_version_suffix} torchvision==0.8.2{torch_version_suffix} -f https://download.pytorch.org/whl/torch_stable.html
!pip install ftfy regex

import torch

print("Torch version:", torch.__version__)

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |███████████████████████         | 834.1MB 1.2MB/s eta 0:04:39tcmalloc: large alloc 1147494400 bytes == 0x5565d83b6000 @  0x7f7690942615 0x55659e20e06c 0x55659e2edeba 0x55659e210e8d 0x55659e30299d 0x55659e284fe9 0x55659e27fb0e 0x55659e21277a 0x55659e284e50 0x55659e27fb0e 0x55659e21277a 0x55659e28186a 0x55659e3037c6 0x55659e280ee2 0x55659e3037c6 0x55659e280ee2 0x55659e3037c6 0x55659e280ee2 0x55659e3037c6 0x55659e385431 0x55659e2e6049 0x55659e250c84 0x55659e2118e9 0x55659e285ade 0x55659e21269a 0x55659e280a45 0x55659e27fe0d 0x55659e21277a 0x55659e280a45 0x55659e21269a 0x55659e280a45
     |█████████████████████████████▏  | 1055.7MB 1.2MB/s eta 0:01:25tcmalloc: large alloc 1434370048 bytes == 0x55661ca0c000 @  0x7f7690942615 0x55659e20e06c 0x55659e2edeba 0x55659e210e8d 0x55659e30299d 0x55659e284fe9 0x55659e27fb0e 0x55659e21277a 0x55659e284e50 0x55659e27fb0e 0x55659e21277a 0x55659e28186a 0x55659e3037c6 0x55659e280ee2 0

In [ ]:
!git clone https://github.com/openai/CLIP.git
%cd CLIP

Cloning into 'CLIP'...
remote: Enumerating objects: 74, done.
remote: Total 74 (delta 0), reused 0 (delta 0), pack-reused 74
Unpacking objects: 100% (74/74), done.
/content/CLIP/CLIP


In [ ]:
import os
import clip
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
main_df.head()

,image_url,label,category,common_in_america
0,/content/images/Bruh/tarta_de_santiago118.jpg,tarta de santiago,cake,cake
1,/content/images/Bruh/bacon76.jpg,bacon,meat dish,bacon
2,/content/images/Bruh/quiche110.jpg,quiche,savory pie,savory pie
3,/content/images/Bruh/cornbread50.jpg,cornbread,bread,cornbread
5,/content/images/Bruh/beef steak58.jpg,beef steak,beef dish,beef dish


In [ ]:
food_types = list(main_df['common_in_america'].unique())
sorted_food = sorted(food_types)
dictionary_key_value = {v:k for k,v in enumerate(sorted_food)}

In [ ]:
labels = list(main_df['common_in_america'])

In [ ]:
list_of_targets = [dictionary_key_value[label] for label in labels]

In [ ]:
main_df['target'] = list_of_targets

In [ ]:
main_df.head()

,image_url,label,category,common_in_america,target
0,/content/images/Bruh/tarta_de_santiago118.jpg,tarta de santiago,cake,cake,15
1,/content/images/Bruh/bacon76.jpg,bacon,meat dish,bacon,0
2,/content/images/Bruh/quiche110.jpg,quiche,savory pie,savory pie,75
3,/content/images/Bruh/cornbread50.jpg,cornbread,bread,cornbread,27
5,/content/images/Bruh/beef steak58.jpg,beef steak,beef dish,beef dish,4


In [ ]:
# https://medium.com/analytics-vidhya/dealing-with-pytorch-custom-datasets-64b6c40fe581
class CustomDatasetFromImages(torch.utils.data.Dataset):
    def __init__(self, dataframe, transform):
        """
        Args:
            csv_path (string): path to csv file
            img_path (string): path to the folder where images are
            transform: pytorch transforms for transforms and tensor conversion
        """
        self.data_info = dataframe
        self.image_arr = np.asarray(self.data_info.iloc[:, 0])
        self.label_arr = np.asarray(self.data_info.iloc[:, 4])
        self.data_len = len(self.data_info.index)
        self.transforms = transform

    def __getitem__(self, index):
        single_image_name = self.image_arr[index]
        img_as_img = Image.open(single_image_name).convert("RGB")
        tensor_image = self.transforms(img_as_img)
        single_image_label = self.label_arr[index]

        return (tensor_image, single_image_label)

    def __len__(self):
        return self.data_len

In [ ]:
import numpy as np
import torch
import clip
from PIL import Image

if torch.cuda.is_available():
  device = "cuda"
else:
  device = "cpu"

model, transform = clip.load("ViT-B/32", device=device) #loading the clip checkpoints
input_resolution = model.input_resolution.item()
context_length = model.context_length.item()
vocab_size = model.vocab_size.item()

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}")
print("Input resolution:", input_resolution)
print("Context length:", context_length)
print("Vocab size:", vocab_size)

100%|████████████████████████████████████████| 354M/354M [00:02<00:00, 162MiB/s]


Model parameters: 151,277,313
Input resolution: 224
Context length: 77
Vocab size: 49408


In [ ]:
from torch.utils.data import DataLoader
import torch
import clip
from PIL import Image
from tqdm import tqdm

if torch.cuda.is_available():
  device = "cuda"
else:
  device = "cpu"

model, preprocess = clip.load("ViT-B/32", device=device) #loading the clip checkpoints
test_dataset = CustomDatasetFromImages(main_df, transform=preprocess)

In [ ]:
torch.manual_seed(42)

def get_features(dataset):
    all_features = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in tqdm(DataLoader(dataset, batch_size=100)):
            features = model.encode_image(images.to(device))

            all_features.append(features)
            all_labels.append(labels)

    return torch.cat(all_features).cpu().numpy(), torch.cat(all_labels).cpu().numpy()

In [ ]:
test_features, test_labels = get_features(test_dataset)

100%|██████████| 244/244 [34:35<00:00,  8.50s/it]


In [ ]:
id_class_name = {k:v for k,v in enumerate(sorted_food)}
class_id = list(id_class_name.keys())
class_name = list(id_class_name.values())
df_id_class = pd.DataFrame({'class_id':class_id,
                            'class_name':class_name})
df_id_class.head()

,class_id,class_name
0,0,bacon
1,1,bagel
2,2,baked potato
3,3,baklava
4,4,beef dish


In [ ]:
text_descriptions = [f"This is a photo of a {label}, a type of food." for label in df_id_class['class_name']]

In [ ]:
texts = clip.tokenize(text_descriptions).to('cpu') 

In [ ]:
def get_topk(texts, test_features, test_labels, top_k=(1,5)):
  with torch.no_grad():
    text_features = model.encode_text(texts).float()
    image_features = test_features
    image_features = torch.from_numpy(image_features)
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)
    _max = max(top_k)
    top_probs, top_labels = text_probs.cpu().topk(_max, dim=-1)
    pred = top_labels.t()
    target = torch.from_numpy(test_labels)
    correct = pred.eq(target.view(1, -1).expand_as(pred))
    res = []
    for k in top_k:
      correct_k = correct[:k].reshape(-1).float().sum(0)
      res.append(correct_k.mul_(100.0 / len(test_labels)))
    inter_tensor = torch.stack(res)
    top1 = inter_tensor.data[0].item()
    top5 = inter_tensor.data[1].item()
    print(f'Top 1 accuracy is {top1}'.format(top1))
    print(f'Top {_max} accuracy is {top5}'.format({_max}, top5))


In [ ]:
get_topk(texts, test_features, test_labels, top_k=(1,2))

Top 1 accuracy is 37.15153884887695
Top 2 accuracy is 51.155723571777344


In [ ]:
get_topk(texts, test_features, test_labels, top_k=(1,3))

Top 1 accuracy is 37.15153884887695
Top 3 accuracy is 59.92527770996094


In [ ]:
get_topk(texts, test_features, test_labels, top_k=(1,5))

Top 1 accuracy is 37.15153884887695
Top 5 accuracy is 70.80510711669922


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(test_features, test_labels, test_size=0.33, 
                                                    random_state=42, stratify=test_labels)

In [ ]:
# from sklearn.model_selection import cross_val_score
# from sklearn.linear_model import LogisticRegression

# #Evaluate L1 regularization strengths for reducing features in final model 
# C = [1, 5, 10, 11, 12, 13, 14] # As C decreases, more coefficients go to zero

# for c in C:
#     clf = LogisticRegression(penalty='l2', C=c, solver='liblinear', class_weight="balanced")
#     # using data before splitting (X_scaled) and (y)
#     scores = cross_val_score(clf, X_train, y_train, cv=5, scoring="accuracy")  #<-- add this
#     print(scores)  #<-- add this

In [ ]:
from sklearn.linear_model import LogisticRegression
# Perform logistic regression
# *Note that the C value should be determined via a hyperparameter sweep using a validation split.
classifier = LogisticRegression(random_state=0, C=14, verbose=1, solver='liblinear', penalty='l2')
classifier.fit(X_train, y_train)

[LibLinear]

LogisticRegression(C=14, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='liblinear', tol=0.0001, verbose=1,
                   warm_start=False)

In [ ]:
def get_topk(classifier, X_test, y_test, top_k = (1, 5)):
  predictions = classifier.predict(X_test)
  probs = classifier.predict_proba(X_test)
  _max = max(top_k)
  best = np.argsort(probs, axis=1)[:,-(_max):]
  pred = torch.from_numpy(np.transpose(best))
  target = torch.from_numpy(y_test)
  correct = pred.eq(target.view(1, -1).expand_as(pred))
  res = []
  for k in top_k:
    correct_k = correct[:k].reshape(-1).float().sum(0)
    res.append(correct_k.mul_(100.0 / len(y_test)))
  inter_tensor = torch.stack(res)
  top5 = inter_tensor.data[1].item()
  top1 = np.mean((y_test == predictions))*100
  print(f'Top 1 accuracy is {top1}'.format(top1))
  print(f'Top {_max} accuracy is {top5}'.format(_max, top5))

In [ ]:
get_topk(classifier, X_test, y_test, top_k = (1,2))

Top 1 accuracy is 74.64543418760886
Top 2 accuracy is 87.27295684814453


In [ ]:
get_topk(classifier, X_test, y_test, top_k = (1, 3))

Top 1 accuracy is 74.64543418760886
Top 3 accuracy is 92.10002899169922


In [ ]:
get_topk(classifier, X_test, y_test, top_k = (1, 5))

Top 1 accuracy is 74.64543418760886
Top 5 accuracy is 95.8322982788086
